In [1]:
from mindnlp.utils import http_get

url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path = http_get(url, './')

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the defa

In [2]:
from mindspore.dataset import TextFileDataset

dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

50000

In [3]:
train_dataset, test_dataset = dataset.split([0.9, 0.1], randomize=False)

In [4]:
# article: [CLS] xxxxx [SEP]
# summary: [CLS] xxxxx [SEP]

In [5]:
import json
import numpy as np

def process_dataset(dataset, tokenizer, batch_size=6, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['input_ids']
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

In [6]:
from mindnlp.transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=4, max_seq_len=300)

In [8]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[4, 300], dtype=Int64, value=
 [[ 101, 1724, 3862 ... 3299, 2419,  102],
  [ 101,  704, 3173 ... 3341, 3975,  102],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]]),
 Tensor(shape=[4, 300], dtype=Int64, value=
 [[ 101, 1724, 3862 ... 3299, 2419,  102],
  [ 101,  704, 3173 ... 3341, 3975,  102],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]])]

In [9]:
len(tokenizer)

21128

In [10]:
from mindnlp.core.nn import functional as F
from mindnlp.transformers import GPT2LMHeadModel

class GPT2ForSummarization(GPT2LMHeadModel):
    def forward(
        self,
        input_ids = None,
        attention_mask = None,
        labels = None,
    ):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        shift_logits = outputs.logits[..., :-1, :]
        shift_labels = labels[..., 1:]
        # Flatten the tokens
        loss = F.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
        return (loss,)

In [11]:
num_epochs = 5
learning_rate = 1.5e-4
warmup_steps = 2000
max_grad_norm = 1.0
num_training_steps = num_epochs * train_dataset.get_dataset_size()

In [12]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=len(tokenizer), bos_token_id=tokenizer.cls_token_id, eos_token_id=tokenizer.pad_token_id)
model = GPT2ForSummarization(config)

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [13]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

number of model parameters: 102068736


In [14]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="gpt2_summarization",
    save_steps=train_dataset.get_dataset_size(),
    save_total_limit=3,
    logging_steps=1000,
    max_steps=num_training_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    warmup_steps=warmup_steps
    
)

In [15]:
from mindnlp.engine import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [16]:
trainer.train()

  0%|                                                                              | 0/56250 [00:00<?, ?it/s]

{'loss': 6.7149, 'learning_rate': 7.5e-05, 'epoch': 0.09}
{'loss': 5.1275, 'learning_rate': 0.00015, 'epoch': 0.18}
{'loss': 4.6146, 'learning_rate': 0.00014723502304147463, 'epoch': 0.27}
{'loss': 4.3565, 'learning_rate': 0.0001444700460829493, 'epoch': 0.36}
{'loss': 4.1458, 'learning_rate': 0.00014170506912442396, 'epoch': 0.44}
{'loss': 4.0208, 'learning_rate': 0.0001389400921658986, 'epoch': 0.53}
{'loss': 3.8927, 'learning_rate': 0.00013617511520737325, 'epoch': 0.62}
{'loss': 3.782, 'learning_rate': 0.00013341013824884793, 'epoch': 0.71}
{'loss': 3.7115, 'learning_rate': 0.00013064516129032258, 'epoch': 0.8}
{'loss': 3.6222, 'learning_rate': 0.00012788018433179722, 'epoch': 0.89}
{'loss': 3.5675, 'learning_rate': 0.0001251152073732719, 'epoch': 0.98}
{'loss': 3.4792, 'learning_rate': 0.00012235023041474655, 'epoch': 1.07}
{'loss': 3.4541, 'learning_rate': 0.00011958525345622118, 'epoch': 1.16}
{'loss': 3.3645, 'learning_rate': 0.00011682027649769584, 'epoch': 1.24}
{'loss': 3.30

TrainOutput(global_step=56250, training_loss=2.9701874880642363, metrics={'train_runtime': 20920.7126, 'train_samples_per_second': 21.51, 'train_steps_per_second': 2.689, 'train_loss': 2.9701874880642363, 'epoch': 5.0})

In [17]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized['input_ids']

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [18]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [19]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

[array([[ 101, 3173, 1290, 5381, 6205, 2128,  128, 3299,  129, 3189, 4510,
        8020, 6381, 5442, 6948, 1132,  840, 8021, 6381, 5442,  129, 3189,
         678, 1286,  794, 7362, 6205, 4209,  689, 1265, 2339, 7415, 1730,
         749, 6237, 1168, 8024,  754,  128, 3299,  127, 3189, 7506, 1814,
        3433, 3409, 1790,  759,  678,  752, 3125,  704, 6158, 1737, 4638,
         125, 1399, 4771, 2339, 2347, 4802, 6371, 1059, 6956, 3647,  767,
         511,  128, 3299,  127, 3189, 1119, 3247,  125, 3198, 8123, 1146,
        8024, 7362, 6205, 4689, 7506, 1814, 3433, 3409, 1790, 4209, 4771,
        2963, 6822,  676, 7339, 1762, 4209, 4771,  122, 1384,  759, 1298,
        7023, 1277,  671, 2339,  868, 7481, 6822, 6121,  868,  689, 3198,
        1355, 4495,  671, 6629, 4209,  680, 4482, 3172, 4960, 1139,  752,
        3125, 8024,  125, 1399, 4771, 2339, 6158, 1737,  511, 3131, 3001,
         782, 1447,  754,  127, 3189,  678, 1286, 1355, 4385,  671, 1399,
        6158, 1737, 4771, 2339, 8024,

In [23]:
model = GPT2LMHeadModel.from_pretrained('./gpt2_summarization/checkpoint-56250', config=config)

In [26]:
model.set_train(False)
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=50, num_beams=5, no_repeat_ngram_size=2)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 10:
        break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[CLS] 新 华 网 西 安 7 月 8 日 电 （ 记 者 郑 凯 伦 ） 记 者 8 日 下 午 从 陕 西 煤 业 化 工 集 团 了 解 到 ， 于 7 月 6 日 韩 城 桑 树 坪 井 下 事 故 中 被 困 的 4 名 矿 工 已 确 认 全 部 死 亡 。 7 月 6 日 凌 晨 4 时 18 分 ， 陕 西 省 韩 城 桑 树 坪 煤 矿 掘 进 三 队 在 煤 矿 1 号 井 南 采 区 一 工 作 面 进 行 作 业 时 发 生 一 起 煤 与 瓦 斯 突 出 事 故 ， 4 名 矿 工 被 困 。 救 援 人 员 于 6 日 下 午 发 现 一 名 被 困 矿 工 ， 经 全 力 抢 救 无 效 死 亡 。 经 救 护 人 员 全 力 救 援 ， 最 后 一 名 遇 难 矿 工 遗 体 已 于 8 日 12 时 36 分 找 到 。 至 此 ， 4 名 遇 难 矿 工 遗 体 全 部 找 到 升 井 ， 救 援 工 作 结 束 。 矿 方 正 在 做 善 后 工 作 。 [SEP] [SEP] 盘 盘 跌 跌 近 跌 30 跌 幅 [SEP] 段 时 跌 长 7 铁 特 来 的 铁 [SEP] 铁 来 铁 铁 单 铁 多 铁 [UNK] 铁 平 铁 交 铁 头 铁 龙 铁 房 铁 四 铁 当 铁 。 铁 香 铁


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[CLS] 参 考 消 息 网 8 月 19 日 报 道 < [UNK] > 外 媒 称 ， 白 宫 聘 雇 了 首 位 正 式 变 性 人 。 拉 菲 · 弗 里 德 曼 - 戈 尔 斯 潘 被 任 命 为 人 事 局 人 员 招 聘 部 主 任 ， 她 此 前 担 任 美 国 变 性 人 平 等 中 心 政 治 问 题 顾 问 。 据 俄 罗 斯 卫 星 网 8 月 19 日 援 引 美 联 社 报 道 ， 中 心 白 宫 解 释 说 ， 他 原 先 的 职 务 " 反 映 了 奥 巴 马 政 府 的 价 值 观 " 。 报 道 称 ， 现 任 美 国 总 统 奥 巴 马 支 持 给 予 变 性 人 与 其 他 美 国 人 平 等 的 权 利 。 早 些 时 候 ， 奥 巴 马 欢 迎 最 美 国 高 法 院 承 认 同 性 婚 姻 ， 之 后 白 宫 亮 起 了 象 征 变 性 人 的 彩 虹 色 。 此 外 ， 最 近 几 个 月 ， 美 国 各 机 构 ， 包 括 军 队 在 内 举 行 支 持 变 性 人 群 体 及 变 性 人 担 任 国 家 机 构 职 位 的 权 利 的 官 方 活 动 。 （ 图 片 来 源 ： 美 联 社 ） [SEP] 5 5 等 等 为 等 因 等 遭 遭 到 讯 等 被 等 名 等 身 等 不 等 原 材 等 或 上 报 价 格 等 物 等 已 不 同 时 的 不 受 理 盘 ， 因 陷 陷 入 深 陷 产 品


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

[CLS] 【 摘 要 】 < [UNK] > 尽 管 凶 手 在 作 案 半 年 后 落 网 ， 但 由 于 尸 体 位 置 隐 蔽 ， 且 深 水 打 捞 难 度 极 大 ， 杭 州 警 方 历 时 2 年 多 先 后 4 次 展 开 搜 捞 行 动 ， 最 终 于 今 年 1 月 6 日 晚 成 功 打 捞 出 被 害 人 遗 体 ， 使 案 件 得 以 成 功 告 破 。 < [UNK] > 第 一 次 在 千 峡 湖 打 捞 尸 体 的 场 景 浙 江 在 线 杭 州 1 月 26 日 讯 ( 浙 江 在 线 记 者 / 俞 雯 [UNK] < [UNK] > 通 讯 员 / 徐 佳 < [UNK] > 胡 学 军 < [UNK] > 首 席 编 辑 / 赵 洁 ) 位 于 丽 水 市 青 田 县 境 内 的 千 峡 湖 ( 滩 坑 水 库 ) ， 是 浙 江 省 内 仅 次 于 千 岛 湖 的 第 二 大 人 工 湖 。 湖 的 上 方 是 有 着 浙 江 第 一 高 桥 之 称 的 北 山 大 桥 ， 大 桥 所 在 湖 面 水 深 达 100 多 米 。 2012 年 8 月 31 日 晚 ， 内 蒙 古 男 子 张 某 被 人 关 进 铁 笼 后 ， 从 这 座 大 桥 上 抛 入 湖 中 ， 从 此 音 讯 全 无 。 尽 管 凶 手 在 作 案 半 年 后 落 网 ， 但 由 于 抛 尸 位 置 隐 蔽 ， 且 深 水 打 捞 难 度 极 大 ， 杭 州 警 方 历 时 2 年 多 先 后 4 次 展 开 搜 捞 行 动 ， 最 终 于 今 年 1 月 6 日 晚 成 功 打 捞 出 被 害 人 遗 体 ， 案 件 得 以 成 功 告 破 。 交 了 近 千 万 赎 金 依 旧 换 不 回 一 条 命 张 某 今 年 47 岁 ， 内 蒙 古 人 ， 在 当 地 开 办 有 大 米 加 工 厂 、 旅 馆 等 多 个 实 业 ， 在 圈 内 小 有 名 气 。 2010 年 ， 张 某 以 企 业 运 作 为 由 向 温 州 男 子 胡 某 借 款 2650 万 元 以 及 其 他 大 额 赌 债 。 2012 年 6 月 10 日 ， 胡 某 邀 请 张 某 到 杭 州 温 德 姆 酒 店 谈 判 还 钱 的 事 。 谈 判 未 果 后 ， 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[CLS] 其 中 一 款 内 地 出 产 的 负 离 子 卫 生 巾 。 ( 图 片 来 源 于 网 络 ) < [UNK] > 参 考 消 息 网 3 月 24 日 报 道 < [UNK] > 港 媒 称 ， 一 批 内 地 制 造 的 卫 生 巾 在 黎 巴 嫩 机 场 被 扣 押 ， 原 因 是 该 批 货 品 被 检 测 出 辐 射 严 重 超 标 。 据 香 港 《 明 报 》 网 站 3 月 23 日 报 道 ， 这 30 箱 卫 生 巾 由 内 地 出 口 ， 20 日 途 径 迪 拜 到 达 黎 巴 嫩 ， 在 贝 鲁 特 [UNK] < [UNK] > [UNK] 国 际 机 场 进 行 扫 描 时 ， 设 备 显 示 箱 中 货 物 辐 射 严 重 超 标 。 测 试 结 果 显 示 ， 这 批 重 达 554 公 斤 的 卫 生 巾 ， 辐 射 量 超 标 了 35 倍 。 货 物 被 扣 押 ， 送 到 黎 巴 嫩 原 子 能 委 员 会 作 进 一 步 的 检 查 。 有 关 当 局 表 示 ， 检 查 了 几 箱 卫 生 巾 后 ， 发 现 它 们 的 辐 射 量 较 黎 巴 嫩 的 标 准 超 标 35 倍 。 而 这 批 卫 生 巾 是 传 说 中 的 [UNK] 负 离 子 卫 生 巾 [UNK] ， 卫 生 巾 公 司 网 站 表 示 ， 棉 网 膜 中 镶 有 负 离 子 芯 片 ， 使 用 时 会 发 射 出 高 浓 度 负 离 子 ， 以 纯 物 理 过 程 实 现 调 节 机 能 、 增 强 免 疫 力 、 消 毒 抗 菌 、 消 除 异 味 的 功 效 ， 亦 无 任 何 副 作 用 。 报 道 称 ， 目 前 还 未 清 楚 辐 射 是 来 否 来 自 卫 生 巾 的 负 离 子 ， 当 地 司 法 机 构 会 介 入 调 查 。 [SEP] 7 [SEP] 盘 盘 贵 贵 铁 铁 贵 ， 贵 6 贵 成 贵 重 贵 表 贵 4 贵 件 贵 珠 贵 情 贵 ： 贵 [UNK] 贵 州 贵 宁 贵 网 贵 阳 贵 7 贵 清 贵 [SEP] 贵 5 贵 2 贵 们


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[CLS] 大 众 网 潍 坊 10 月 13 日 讯 （ 记 者 < [UNK] > 单 姗 < [UNK] > 通 讯 员 < [UNK] > 周 显 江 < [UNK] > 赵 力 智 ） 日 前 ， 诸 城 四 少 年 沉 溺 于 网 络 和 电 玩 游 戏 ， 效 仿 暴 力 电 子 游 戏 情 景 ， 白 天 骑 摩 托 车 抢 劫 寻 求 刺 激 ， 年 龄 最 大 的 刘 某 只 有 19 岁 ， 其 余 三 人 年 仅 13 岁 。 四 少 年 白 天 骑 摩 托 车 抢 劫 < [UNK] > 9 月 22 日 中 午 12 点 30 分 ， 潍 坊 诸 城 市 石 桥 子 镇 臧 某 和 孙 某 骑 电 动 车 行 驶 至 诸 城 市 石 桥 子 镇 前 浩 [UNK] 村 前 ， 突 然 有 两 辆 摩 托 车 从 后 面 赶 上 ， 把 臧 某 和 孙 某 逼 到 路 边 停 下 。 < [UNK] > 从 两 辆 摩 托 车 上 下 来 四 名 少 年 ， 两 名 少 年 手 持 棒 球 棒 ， 一 名 少 年 手 持 伸 缩 鞭 ， 四 名 少 年 让 臧 某 和 孙 某 把 钱 和 手 机 交 出 来 。 臧 某 还 以 为 四 名 少 年 开 玩 笑 ， 就 笑 着 说 不 要 开 玩 笑 了 自 己 还 有 事 ， 见 臧 某 没 把 自 己 放 在 眼 里 ， 其 中 一 少 年 发 怒 了 ， 手 持 棒 球 棒 将 臧 某 的 电 动 车 前 篮 打 破 。 < [UNK] > 臧 某 和 孙 某 这 才 意 识 到 ， 这 四 名 少 年 不 是 开 玩 笑 而 是 真 抢 劫 ， 他 俩 把 手 机 和 1000 元 现 金 乖 乖 地 交 出 ， 四 少 年 拿 到 手 机 和 现 金 后 骑 摩 托 车 扬 长 而 去 。 < [UNK] > 民 警 走 访 途 中 抓 获 两 抢 劫 少 年 10 月 9 日 下 午 5 点 10 分 ， 诸 城 市 公 安 局 石 桥 子 派 出 所 教 导 员 王 金 吉 、 副 所 长 卢 金 生 、 协 勤 人 员 付 斌 到 诸 城 市 石 桥 子 镇 大 朱 苏 铺 社 区 走 访 途 中 ， 发 现 一 辆 白 色 250 摩 托 车 停 在 路 边 ， 车 边 有 